In [1]:
# Import & install libraries
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.preprocessing import *
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import accuracy_score, f1_score

In [2]:
!pip install PySastrawi
!pip install scikit-multilearn

     |████████████████████████████████| 210 kB 4.2 MB/s 
     |████████████████████████████████| 89 kB 3.2 MB/s 


In [3]:
# Loading dataset CSV into Dataframe
contoh = pd.read_csv('/content/drive/MyDrive/Data TK 1 Text Mining/clean/contoh_data.csv')
human_ant = pd.read_csv('/content/drive/MyDrive/Data TK 1 Text Mining/clean/human_annotated.csv')
machine_ant = pd.read_csv('/content/drive/MyDrive/Data TK 1 Text Mining/clean/machine_annotated.csv')
testing = pd.read_csv('/content/drive/MyDrive/Data TK 1 Text Mining/clean/testing_data.csv')
unlabeled = pd.read_csv('/content/drive/MyDrive/Data TK 1 Text Mining/clean/unlabeled_data.csv')

In [4]:
training_data = machine_ant.append(human_ant, ignore_index = True)
training_data = training_data.append(contoh, ignore_index = True)

In [5]:
# Normalizing text & removing stopwords
import re

def normalize(text):
    res = text.lower().strip() # Mengubah uppercase menjadi lowercase dan melakukan trimming pada teks
    res = re.sub("\d", "", res) # Menghilangkan angka
    res = re.sub("\s+", " ", res) # Menghilangkan spasi berlebih
    res = re.sub("[^\w\s]", "", res) # Menghilangkan tanda baca
    return res

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

stop_factory = StopWordRemoverFactory()
lst_stopword = stop_factory.get_stop_words()

# Fungsi untuk menghilangkan stopword dari suatu teks

def remove_stopword(text):
    lst_token = text.split(" ")
    res = []
    for token in lst_token:
        if token not in lst_stopword:
            res.append(token)
    return " ".join(res)

def clean_words(arr):
  corpus = []
  for text in arr:
    normalized_text = normalize(text)
    cleaned_text = remove_stopword(normalized_text)
    corpus.append(cleaned_text)
  return corpus
def replace_with_cleaned(arr,arr_clean):
  for i in range(len(arr_clean)):
    arr[i] = arr_clean[i]

In [6]:
training_isi = training_data['isi'].astype(str)
training_clean = clean_words(training_isi)

In [7]:
testing_isi = testing['isi'].astype(str)
testing_clean = clean_words(testing_isi)

In [8]:
replace_with_cleaned(training_data['isi'],training_clean)
replace_with_cleaned(testing['isi'],testing_clean)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:
training_data['isi'] = training_data['judul']+training_data['isi']

In [10]:
testing['isi'] = testing['judul']+testing['isi']

In [11]:
testing_data = testing.copy()
testing_data

,id,judul,isi
0,DS-45,kenapa telinga berdengung tidak bisa hilang ?,kenapa telinga berdengung tidak bisa hilang ?s...
1,DS-339,apabila anak anda lambat berbicara,apabila anak anda lambat berbicarakenal team d...
2,AD-18,sakit perut setiap habis makan dan minum,sakit perut setiap habis makan dan minumselama...
3,AD-66,cara mengobati vertigo berkepanjangan,cara mengobati vertigo berkepanjanganselamat m...
4,AD-73,imunisasi dapat dan polio,imunisasi dapat dan poliomalam dok anak umur ...
...,...,...,...
2922,KD-43404,keluhan setelah berhubungan,keluhan setelah berhubungandok minggu bersetub...
2923,KD-43406,ingin menambah tinggi badan,ingin menambah tinggi badanselamat malam dok ...
2924,KD-43411,anak 1 tahun boleh dikasih nasi ?,anak 1 tahun boleh dikasih nasi ?bayi berumur ...
2925,KD-43446,mencegah infeksi virus zika,mencegah infeksi virus zikaselamat siang dok b...


In [12]:
training_data

,id,label_0,label_1,label_2,Unnamed: 4,Unnamed: 5,label_3,label_4,label_5,label_6,label_7,label_8,judul,isi
0,AD-10,Kesehatan Kulit dan Kelamin,Umum,Urologi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gatal-gatal dan bintik merah di kelamin,gatal-gatal dan bintik merah di kelamindok cow...
1,AD-15,Bedah,Urologi,Penyakit Dalam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sering buang air kecil,sering buang air kecilselamat siang dok mingg...
2,AD-28,Jantung dan Pembuluh Darah,Penyakit Dalam,Umum,NaN,NaN,Bedah,NaN,NaN,NaN,NaN,NaN,penyebab nyeri dada,penyebab nyeri dadaassalammu alaikum sejahter...
3,AD-33,Penyakit Dalam,Saraf,Kesehatan Anak,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,penanganan luka digigit anjing,penanganan luka digigit anjingmalam dok digig...
4,AD-36,Kesehatan Jiwa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,insomnia karena banyak pikiran,insomnia karena banyak pikiranmalam dok knlin...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13308,AD-15274,Kebidanan dan Kandungan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pendarahan pasca keguguran,pendarahan pasca keguguranmengalami menstruasi...
13309,AD-15296,Bedah,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ambein eksternal,ambein eksternaldok berusia tahun lakukan me...
13310,AD-15362,"Kesehatan Telinga, Hidung dan Tenggorokan (THT)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,apa ini polip ?,apa ini polip ?dok meri akhirakhir merasakan ...
13311,AD-15607,Gigi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gusi depan tumbuh daging lebih,gusi depan tumbuh daging lebihdokgusi bwah tum...


In [13]:
X_train = training_data['isi']
X_test = testing_data['isi']

In [14]:
#Word Embedding using Word2Vec

# Class untuk enkapsulasi beberapa fungsionalitas yang ada

from gensim import models

class Word2VecModel:
    #Yoga
    # pretrained_file = "/content/drive/MyDrive/Colab Notebooks/Data Mining/Data TK 1 Text Mining/word2vec.id.300d.txt.gz"
    #Amanda
    pretrained_file = "/content/drive/MyDrive/Data TK 1 Text Mining/word2vec.id.300d.txt.gz"
    vector_size = 300
    model = models.KeyedVectors.load_word2vec_format(pretrained_file)

    # Fungsi untuk mencari representasi vektor dari suatu kata  
    def get_word_embedding(self, word):
        try:
            return self.model[word]
        except: # Melakukan handling terhadap OOV
            return np.nan
    
    # Fungsi untuk mencari representasi vektor dari suatu kalimat
    def get_sentence_embedding(self, sent):
        lst_token = re.findall("[^\s]{3,}", sent.lower())
        rep = []
        for token in lst_token:
            try:
                token_rep = self.model[token]
                rep.append(token_rep)
            except: # Melakukan handling terhadap OOV
                continue
        return np.mean(rep, axis=0)
    
    # Fungsi untuk mengubah setiap teks pada suatu corpus menjadi representasi vektornya
    def transform(self, corpus):
        res = []
        for doc in corpus:
            tmp = self.get_sentence_embedding(doc)
            if type(tmp) == np.float64: # Melakukan handling terhadap OOV
                tmp = np.zeros(self.vector_size)
            res.append(tmp)
        return np.array(res)

In [15]:
model_word2vec = Word2VecModel()

In [16]:
# Vektorisasi X
X_train_w2v = model_word2vec.transform(X_train)
X_test_w2v = model_word2vec.transform(X_test)

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [17]:
X_train_w2v

array([[-0.15749151,  0.43431121, -0.80271339, ...,  0.00734386,
        -0.2319068 ,  0.37142077],
       [ 0.21709844,  0.89783949, -0.30415702, ..., -0.10554989,
         0.13633913, -0.11480051],
       [-0.03204537,  0.38140777, -1.19352901, ..., -0.10880705,
         0.03331447,  0.50040686],
       ...,
       [ 0.41515628,  0.34865534, -0.20984215, ..., -0.132357  ,
         0.27561939,  0.02290982],
       [-0.66094965,  0.4288193 , -1.26529229, ..., -0.04095858,
        -0.04814701, -0.12124054],
       [-0.0117085 ,  1.47356319, -0.7892819 , ...,  0.01145591,
        -0.30869934,  0.26579189]])

In [18]:
X_test_w2v

array([[ 0.6297807 ,  0.23537913, -0.8162581 , ..., -0.36713877,
         0.15409379,  0.2060279 ],
       [-0.01053726,  0.8196182 , -1.2517612 , ...,  0.27441123,
        -0.45853963,  0.37417328],
       [-0.2455704 ,  0.4529342 , -0.4602846 , ...,  0.3283963 ,
        -0.08191991,  0.14049804],
       ...,
       [-0.49590194,  0.40556112, -0.34396315, ...,  1.2069584 ,
        -0.7541469 ,  0.41678   ],
       [ 0.69754857,  1.3868108 , -0.1501423 , ..., -0.01128477,
         0.24882856, -0.09585685],
       [ 0.62760043,  0.8952309 , -1.1857886 , ..., -0.26900423,
         0.03051018,  0.18986353]], dtype=float32)

In [19]:
y_train = training_data.copy()
y_train.drop(columns=['id','judul','isi'], inplace=True)

In [20]:
KebidananArray = []
for index, row in y_train.iterrows():
  if row['label_0'] == "Kebidanan dan Kandungan" or row['label_1'] == "Kebidanan dan Kandungan" or row['label_2'] == "Kebidanan dan Kandungan"or row['label_3'] == "Kebidanan dan Kandungan"or row['label_4'] == "Kebidanan dan Kandungan"or row['label_5'] == "Kebidanan dan Kandungan"or row['label_6'] == "Kebidanan dan Kandungan"or row['label_7'] == "Kebidanan dan Kandungan"or row['label_8'] == "Kebidanan dan Kandungan":
    KebidananArray.append("1")
  else:
    KebidananArray.append("0")
print(len(KebidananArray))

13313


In [21]:
DalamArray = []
for index, row in y_train.iterrows():
  if row['label_0'] == "Penyakit Dalam" or row['label_1'] == "Penyakit Dalam" or row['label_2'] == "Penyakit Dalam" or row['label_3'] == "Penyakit Dalam" or row['label_4'] == "Penyakit Dalam" or row['label_5'] == "Penyakit Dalam" or row['label_6'] == "Penyakit Dalam" or row['label_7'] == "Penyakit Dalam" or row['label_8'] == "Penyakit Dalam":
    DalamArray.append("1")
  else:
    DalamArray.append("0")
print(len(DalamArray))

13313


In [22]:
AnakArray = []
for index, row in y_train.iterrows():
  if row['label_0'] == "Kesehatan Anak" or row['label_1'] == "Kesehatan Anak" or row['label_2'] == "Kesehatan Anak" or row['label_3'] == "Kesehatan Anak" or row['label_4'] == "Kesehatan Anak" or row['label_5'] == "Kesehatan Anak" or row['label_6'] == "Kesehatan Anak" or row['label_7'] == "Kesehatan Anak" or row['label_8'] == "Kesehatan Anak":
    AnakArray.append("1")
  else:
    AnakArray.append("0")

KulitArray = []
for index, row in y_train.iterrows():
  if row['label_0'] == "Kesehatan Kulit dan Kelamin" or row['label_1'] == "Kesehatan Kulit dan Kelamin" or row['label_2'] == "Kesehatan Kulit dan Kelamin" or row['label_3'] == "Kesehatan Kulit dan Kelamin" or row['label_4'] == "Kesehatan Kulit dan Kelamin" or row['label_5'] == "Kesehatan Kulit dan Kelamin"or row['label_6'] == "Kesehatan Kulit dan Kelamin" or row['label_7'] == "Kesehatan Kulit dan Kelamin" or row['label_8'] == "Kesehatan Kulit dan Kelamin":
    KulitArray.append("1")
  else:
    KulitArray.append("0")

GiziArray = []
for index, row in y_train.iterrows():
  if row['label_0'] == "Kesehatan Gizi" or row['label_1'] == "Kesehatan Gizi" or row['label_2'] == "Kesehatan Gizi"or row['label_3'] == "Kesehatan Gizi" or row['label_4'] == "Kesehatan Gizi"or row['label_5'] == "Kesehatan Gizi" or row['label_6'] == "Kesehatan Gizi"or row['label_7'] == "Kesehatan Gizi" or row['label_8'] == "Kesehatan Gizi":
    GiziArray.append("1")
  else:
    GiziArray.append("0")

THTArray = []
for index, row in y_train.iterrows():
  if row['label_0'] == "Kesehatan Telinga, Hidung dan Tenggorokan (THT)" or row['label_1'] == "Kesehatan Telinga, Hidung dan Tenggorokan (THT)" or row['label_2'] == "Kesehatan Telinga, Hidung dan Tenggorokan (THT)"or row['label_3'] == "Kesehatan Telinga, Hidung dan Tenggorokan (THT)" or row['label_4'] == "Kesehatan Telinga, Hidung dan Tenggorokan (THT)"or row['label_5'] == "Kesehatan Telinga, Hidung dan Tenggorokan (THT)" or row['label_6'] == "Kesehatan Telinga, Hidung dan Tenggorokan (THT)"or row['label_7'] == "Kesehatan Telinga, Hidung dan Tenggorokan (THT)" or row['label_8'] == "Kesehatan Telinga, Hidung dan Tenggorokan (THT)":
    THTArray.append("1")
  else:
    THTArray.append("0")

GigiArray = []
for index, row in y_train.iterrows():
  if row['label_0'] == "Gigi" or row['label_1'] == "Gigi" or row['label_2'] == "Gigi"or row['label_3'] == "Gigi" or row['label_4'] == "Gigi"or row['label_5'] == "Gigi" or row['label_6'] == "Gigi"or row['label_7'] == "Gigi" or row['label_8'] == "Gigi":
    GigiArray.append("1")
  else:
    GigiArray.append("0")

MataArray = []
for index, row in y_train.iterrows():
  if row['label_0'] == "Kesehatan Mata" or row['label_1'] == "Kesehatan Mata" or row['label_2'] == "Kesehatan Mata"or row['label_3'] == "Kesehatan Mata" or row['label_4'] == "Kesehatan Mata"or row['label_5'] == "Kesehatan Mata" or row['label_6'] == "Kesehatan Mata"or row['label_7'] == "Kesehatan Mata" or row['label_8'] == "Kesehatan Mata":
    MataArray.append("1")
  else:
    MataArray.append("0")

BedahArray = []
for index, row in y_train.iterrows():
  if row['label_0'] == "Bedah" or row['label_1'] == "Bedah" or row['label_2'] == "Bedah"or row['label_3'] == "Bedah" or row['label_4'] == "Bedah"or row['label_5'] == "Bedah" or row['label_6'] == "Bedah"or row['label_7'] == "Bedah" or row['label_8'] == "Bedah":
    BedahArray.append("1")
  else:
    BedahArray.append("0")

KesehatanJiwaArray = []
for index, row in y_train.iterrows():
  if row['label_0'] == "Kesehatan Jiwa" or row['label_1'] == "Kesehatan Jiwa" or row['label_2'] == "Kesehatan Jiwa"or row['label_3'] == "Kesehatan Jiwa" or row['label_4'] == "Kesehatan Jiwa"or row['label_5'] == "Kesehatan Jiwa" or row['label_6'] == "Kesehatan Jiwa"or row['label_7'] == "Kesehatan Jiwa" or row['label_8'] == "Kesehatan Jiwa":
    KesehatanJiwaArray.append("1")
  else:
    KesehatanJiwaArray.append("0")

TulangArray = []
for index, row in y_train.iterrows():
  if row['label_0'] == "Ortopedi (Tulang)" or row['label_1'] == "Ortopedi (Tulang)" or row['label_2'] == "Ortopedi (Tulang)"or row['label_3'] == "Ortopedi (Tulang)" or row['label_4'] == "Ortopedi (Tulang)"or row['label_5'] == "Ortopedi (Tulang)" or row['label_6'] == "Ortopedi (Tulang)"or row['label_7'] == "Ortopedi (Tulang)" or row['label_8'] == "Ortopedi (Tulang)":
    TulangArray.append("1")
  else:
    TulangArray.append("0")

JantungArray = []
for index, row in y_train.iterrows():
  if row['label_0'] == "Jantung dan Pembuluh Darah" or row['label_1'] == "Jantung dan Pembuluh Darah" or row['label_2'] == "Jantung dan Pembuluh Darah" or row['label_3'] == "Jantung dan Pembuluh Darah" or row['label_4'] == "Jantung dan Pembuluh Darah" or row['label_5'] == "Jantung dan Pembuluh Darah" or row['label_6'] == "Jantung dan Pembuluh Darah" or row['label_7'] == "Jantung dan Pembuluh Darah" or row['label_8'] == "Jantung dan Pembuluh Darah":
    JantungArray.append("1")
  else:
    JantungArray.append("0")

UrologiArray = []
for index, row in y_train.iterrows():
  if row['label_0'] == "Urologi" or row['label_1'] == "Urologi" or row['label_2'] == "Urologi"or row['label_3'] == "Urologi" or row['label_4'] == "Urologi"or row['label_5'] == "Urologi" or row['label_6'] == "Urologi"or row['label_7'] == "Urologi" or row['label_8'] == "Urologi":
    UrologiArray.append("1")
  else:
    UrologiArray.append("0")

SarafArray = []
for index, row in y_train.iterrows():
  if row['label_0'] == "Saraf" or row['label_1'] == "Saraf" or row['label_2'] == "Saraf"or row['label_3'] == "Saraf" or row['label_4'] == "Saraf"or row['label_5'] == "Saraf" or row['label_6'] == "Saraf"or row['label_7'] == "Saraf" or row['label_8'] == "Saraf":
    SarafArray.append("1")
  else:
    SarafArray.append("0")

ParuArray = []
for index, row in y_train.iterrows():
  if row['label_0'] == "Pulmonologi (Paru)" or row['label_1'] == "Pulmonologi (Paru)" or row['label_2'] == "Pulmonologi (Paru)"or row['label_3'] == "Pulmonologi (Paru)" or row['label_4'] == "Pulmonologi (Paru)"or row['label_5'] == "Pulmonologi (Paru)" or row['label_6'] == "Pulmonologi (Paru)"or row['label_7'] == "Pulmonologi (Paru)" or row['label_8'] == "Pulmonologi (Paru)":
    ParuArray.append("1")
  else:
    ParuArray.append("0")

UmumArray = []
for index, row in y_train.iterrows():
  if row['label_0'] == "Umum" or row['label_1'] == "Umum" or row['label_2'] == "Umum" or row['label_3'] == "Umum" or row['label_4'] == "Umum"or row['label_5'] == "Umum" or row['label_6'] == "Umum"or row['label_7'] == "Umum" or row['label_8'] == "Umum":
    UmumArray.append("1")
  else:
    UmumArray.append("0")

In [23]:
y = [KebidananArray, DalamArray, AnakArray, KulitArray, GiziArray, THTArray, GigiArray, MataArray, BedahArray, KesehatanJiwaArray, TulangArray, JantungArray, UrologiArray, SarafArray, ParuArray, UmumArray]

y_train["KebidananArray"] = KebidananArray
y_train["DalamArray"] = DalamArray
y_train["AnakArray"] = AnakArray
y_train["KulitArray"] = KulitArray
y_train["GiziArray"] = GiziArray
y_train["THTArray"] = THTArray
y_train["GigiArray"] = GigiArray
y_train["MataArray"] = MataArray
y_train["BedahArray"] = BedahArray
y_train["KesehatanJiwaArray"] = KesehatanJiwaArray
y_train["TulangArray"] = TulangArray
y_train["JantungArray"] = JantungArray
y_train["UrologiArray"] = UrologiArray
y_train["SarafArray"] = SarafArray
y_train["ParuArray"] = ParuArray
y_train["UmumArray"] = UmumArray

In [24]:
y_train_encoded = y_train.copy()
y_train_encoded.drop(columns=['label_0','label_1','label_2','label_3',
                              'label_4','label_5','label_6','label_7',
                              'label_8','Unnamed: 4','Unnamed: 5'], inplace=True)

In [25]:
y_train_encoded

,KebidananArray,DalamArray,AnakArray,KulitArray,GiziArray,THTArray,GigiArray,MataArray,BedahArray,KesehatanJiwaArray,TulangArray,JantungArray,UrologiArray,SarafArray,ParuArray,UmumArray
0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1
1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0
2,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1
3,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13308,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13309,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
13310,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
13311,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [26]:
# Binary Relevance
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB

# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
classifier = BinaryRelevance(GaussianNB())

# train
classifier.fit(X_train_w2v, y_train_encoded.astype(int))

# predict
predictions = classifier.predict(X_test_w2v)

In [27]:
testing_duplicate = testing_data.iloc[:,[0]]
testing_duplicate

,id
0,DS-45
1,DS-339
2,AD-18
3,AD-66
4,AD-73
...,...
2922,KD-43404
2923,KD-43406
2924,KD-43411
2925,KD-43446


In [28]:
pred_arr = predictions.toarray()
pred_arr[0]

array([0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1], dtype=int64)

In [29]:
testing_duplicate['Kebidanan'] = pred_arr[:,[0]]
testing_duplicate['Dalam'] = pred_arr[:,[1]]
testing_duplicate['Anak'] = pred_arr[:,[2]]
testing_duplicate['Kulit'] = pred_arr[:,[3]]
testing_duplicate['Gizi'] = pred_arr[:,[4]]
testing_duplicate['THT'] = pred_arr[:,[5]]

testing_duplicate['Gigi'] = pred_arr[:,[6]]
testing_duplicate['Mata'] = pred_arr[:,[7]]
testing_duplicate['Bedah'] = pred_arr[:,[8]]
testing_duplicate['KesehatanJiwa'] = pred_arr[:,[9]]

testing_duplicate['Tulang'] = pred_arr[:,[10]]
testing_duplicate['Jantung'] = pred_arr[:,[11]]
testing_duplicate['Urologi'] = pred_arr[:,[12]]
testing_duplicate['Saraf'] = pred_arr[:,[13]]
testing_duplicate['Paru'] = pred_arr[:,[14]]

testing_duplicate['Umum'] = pred_arr[:,[15]]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [30]:
testing_duplicate

,id,Kebidanan,Dalam,Anak,Kulit,Gizi,THT,Gigi,Mata,Bedah,KesehatanJiwa,Tulang,Jantung,Urologi,Saraf,Paru,Umum
0,DS-45,0,1,0,1,0,1,1,0,1,0,0,0,1,1,1,1
1,DS-339,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0
2,AD-18,1,1,1,1,0,1,0,0,1,0,0,1,1,0,1,1
3,AD-66,1,1,0,1,0,1,0,0,1,1,0,1,1,1,1,1
4,AD-73,1,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2922,KD-43404,1,1,0,1,0,1,0,0,1,0,1,1,1,1,1,1
2923,KD-43406,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2924,KD-43411,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
2925,KD-43446,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [31]:
testing_duplicate.to_csv('/content/drive/MyDrive/Data TK 1 Text Mining/problem transformation/binary_relevance_preds.csv', index=False, header=False)

In [32]:
# using Classifier Chains
from skmultilearn.problem_transform import ClassifierChain
from sklearn.naive_bayes import GaussianNB

# initialize classifier chains multi-label classifier
# with a gaussian naive bayes base classifier
cc_classifier = ClassifierChain(GaussianNB())

# train
cc_classifier.fit(X_train_w2v, y_train_encoded.astype(int))

# predict
pred_cc = cc_classifier.predict(X_test_w2v)

In [33]:
testing_cc = testing_data.iloc[:,[0]]

In [34]:
pred_cc_arr = pred_cc.toarray()

In [35]:
testing_cc['Kebidanan'] = pred_cc_arr[:,[0]].astype(int)
testing_cc['Dalam'] = pred_cc_arr[:,[1]].astype(int)
testing_cc['Anak'] = pred_cc_arr[:,[2]].astype(int)
testing_cc['Kulit'] = pred_cc_arr[:,[3]].astype(int)
testing_cc['Gizi'] = pred_cc_arr[:,[4]].astype(int)
testing_cc['THT'] = pred_cc_arr[:,[5]].astype(int)

testing_cc['Gigi'] = pred_cc_arr[:,[6]].astype(int)
testing_cc['Mata'] = pred_cc_arr[:,[7]].astype(int)
testing_cc['Bedah'] = pred_cc_arr[:,[8]].astype(int)
testing_cc['KesehatanJiwa'] = pred_cc_arr[:,[9]].astype(int)

testing_cc['Tulang'] = pred_cc_arr[:,[10]].astype(int)
testing_cc['Jantung'] = pred_cc_arr[:,[11]].astype(int)
testing_cc['Urologi'] = pred_cc_arr[:,[12]].astype(int)
testing_cc['Saraf'] = pred_cc_arr[:,[13]].astype(int)
testing_cc['Paru'] = pred_cc_arr[:,[14]].astype(int)

testing_cc['Umum'] = pred_cc_arr[:,[15]].astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
testing_cc

,id,Kebidanan,Dalam,Anak,Kulit,Gizi,THT,Gigi,Mata,Bedah,KesehatanJiwa,Tulang,Jantung,Urologi,Saraf,Paru,Umum
0,DS-45,0,1,0,1,0,1,0,0,1,0,0,0,0,0,1,1
1,DS-339,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
2,AD-18,1,1,0,1,0,0,0,0,1,0,1,0,1,0,0,1
3,AD-66,1,1,0,1,0,0,0,0,1,0,1,0,1,0,0,1
4,AD-73,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2922,KD-43404,1,1,0,1,0,0,0,0,1,0,1,0,1,0,0,1
2923,KD-43406,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2924,KD-43411,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2925,KD-43446,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [37]:
testing_cc.to_csv('/content/drive/MyDrive/Data TK 1 Text Mining/problem transformation/classifier_chains_preds.csv', index=False, header=False)

In [38]:
# using Label Powerset
from skmultilearn.problem_transform import LabelPowerset
from sklearn.naive_bayes import GaussianNB

# initialize Label Powerset multi-label classifier
# with a gaussian naive bayes base classifier
lp_classifier = LabelPowerset(GaussianNB())

# train
lp_classifier.fit(X_train_w2v, y_train_encoded.astype(int))

# predict
pred_lp = lp_classifier.predict(X_test_w2v)

In [39]:
testing_lp = testing_data.iloc[:,[0]]
pred_lp_arr = pred_lp.toarray()

In [40]:
testing_lp['Kebidanan'] = pred_lp_arr[:,[0]]
testing_lp['Dalam'] = pred_lp_arr[:,[1]]
testing_lp['Anak'] = pred_lp_arr[:,[2]]
testing_lp['Kulit'] = pred_lp_arr[:,[3]]
testing_lp['Gizi'] = pred_lp_arr[:,[4]]
testing_lp['THT'] = pred_lp_arr[:,[5]]

testing_lp['Gigi'] = pred_lp_arr[:,[6]]
testing_lp['Mata'] = pred_lp_arr[:,[7]]
testing_lp['Bedah'] = pred_lp_arr[:,[8]]
testing_lp['KesehatanJiwa'] = pred_lp_arr[:,[9]]

testing_lp['Tulang'] = pred_lp_arr[:,[10]]
testing_lp['Jantung'] = pred_lp_arr[:,[11]]
testing_lp['Urologi'] = pred_lp_arr[:,[12]]
testing_lp['Saraf'] = pred_lp_arr[:,[13]]
testing_lp['Paru'] = pred_lp_arr[:,[14]]

testing_lp['Umum'] = pred_lp_arr[:,[15]]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [41]:
testing_lp.to_csv('/content/drive/MyDrive/Data TK 1 Text Mining/problem transformation/label_powerset_preds.csv', index=False, header=False)